In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.6.tgz
!tar xf spark-2.4.7-bin-hadoop2.6.tgz
!pip install -q findspark

In [3]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/
                        
Using Scala version 2.11.8, OpenJDK 64-Bit Server VM, 11.0.9
Branch 
Compiled by user jshao on 2018-09-16T11:51:52Z
Revision 
Url 
Type --help for more information.


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.6"

In [5]:
from google.colab import drive
drive.mount("/content/drive") 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [7]:
!pip install pyspark==2.3.2

In [8]:
!pyspark --packages graphframes:graphframes:0.7.0-spark2.3-s_2.11 --jars graphframes-0.7.0-spark2.3-s_2.11.jar

Python 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/content/spark-2.4.7-bin-hadoop2.6/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0438a41e-0a2c-4241-b842-8b8d45690361;1.0
	confs: [default]
	found graphframes#graphframes;0.7.0-spark2.3-s_2.11 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 247ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.7.0-spark2.3-s_2.11 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            || 

In [9]:
!cp /root/.ivy2/jars/* /content/spark-2.4.7-bin-hadoop2.6/jars

In [10]:
!pyspark --packages graphframes:graphframes:0.7.0-spark2.3-s_2.11 --jars graphframes-0.7.0-spark2.3-s_2.11.jar

Python 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/content/spark-2.4.7-bin-hadoop2.6/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2b2b1685-75b5-4ce2-8257-d99757febba5;1.0
	confs: [default]
	found graphframes#graphframes;0.7.0-spark2.3-s_2.11 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 353ms :: artifacts dl 6ms
	:: modules in use:
	graphframes#graphframes;0.7.0-spark2.3-s_2.11 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            || 

In [11]:
import pyspark
import pandas as pd

In [ ]:
# sc = pyspark.SparkContext(appName="GraphApp") 

In [12]:
df = pd.read_csv('/content/drive/My Drive/Twitter-Dataset-Analysis/295B/Nov09/user-follower-weighted_100_Nov09.csv')

In [13]:
df.head(5)

,user_id,weights,followers
0,1084.0,0.65,8.207108e+17
1,1084.0,0.65,5.692212e+06
2,1084.0,0.65,2.892513e+07
3,1084.0,0.65,2.384755e+07
4,1084.0,0.65,1.189936e+07


In [14]:
nodes = []
for index, row in df.iterrows():
        nodes.append(row['user_id'])
        nodes.append(row['followers'])

In [15]:
nodes = list(set(nodes))

In [16]:
df = pd.DataFrame (nodes,columns=['Nodes'])

In [17]:
df['id'] = df.index

In [18]:
df.head(5)

,Nodes,id
0,3.748659e+09,0
1,3.068658e+09,1
2,2.237661e+09,2
3,3.717202e+08,3
4,1.137705e+08,4


In [19]:
df[df['id'] == 1688]

,Nodes,id
1688,87035829.0,1688


In [ ]:
# df.to_csv('/content/drive/My Drive/Twitter-Dataset-Analysis/295B/Nodes_table.csv',index=False)

In [ ]:
# df = spark.createDataFrame(df)

In [20]:
main_df = pd.read_csv('/content/drive/My Drive/Twitter-Dataset-Analysis/295B/Nov09/user-follower-weighted_100_Nov09.csv')

In [21]:
# main_df['relationship'] = 'Followed by'
main_df['relationship'] = main_df['weights']

In [22]:
main_df = main_df.drop(columns=['weights'])


In [23]:
main_df = main_df.rename(columns={'user_id':'src','followers':'dst'})
main_df

,src,dst,relationship
0,1084.0,8.207108e+17,0.65
1,1084.0,5.692212e+06,0.65
2,1084.0,2.892513e+07,0.65
3,1084.0,2.384755e+07,0.65
4,1084.0,1.189936e+07,0.65
...,...,...,...
243748,20196271.0,7.366754e+07,0.55
243749,20196271.0,3.631694e+07,0.55
243750,20196271.0,6.398302e+07,0.55
243751,20196271.0,7.040932e+06,0.55


In [ ]:
main_df.to_csv('/content/drive/My Drive/Twitter-Dataset-Analysis/295B/Edges_table.csv',index=False)

In [ ]:
# main_df= spark.createDataFrame(main_df)

In [24]:
!pip install graphframes

In [25]:
from graphframes import GraphFrame

In [26]:
from pyspark.sql import SQLContext 
sqlCtx = SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)


In [27]:
sdf = sqlCtx.createDataFrame(df)
mdf = sqlCtx.createDataFrame(main_df)

In [28]:
g = GraphFrame(sdf, mdf)

In [29]:
## Take a look at the DataFrames
g.vertices.show()

+--------------------+---+
|               Nodes| id|
+--------------------+---+
|         3.7486592E9|  0|
|       3.068657669E9|  1|
|       2.237661193E9|  2|
|        3.71720202E8|  3|
|        1.13770512E8|  4|
|                17.0|  5|
|        1.14819092E8|  6|
|          6.553602E7|  7|
|         5.5050265E7|  8|
|         6.2914587E7|  9|
|         3.6700187E7| 10|
|         1.4680095E7| 11|
|        2.46415407E8| 12|
|           8388662.0| 13|
|       2.353528897E9| 14|
|       1.598029891E9| 15|
|       2.349858888E9| 16|
|9.042924065264352E17| 17|
|7.889592560537477...| 18|
|           1048651.0| 19|
+--------------------+---+
only showing top 20 rows



In [30]:
g.edges.show()


+------+--------------------+------------------+
|   src|                 dst|      relationship|
+------+--------------------+------------------+
|1084.0|8.207108245955584E17|0.6500000000000001|
|1084.0|           5692212.0|0.6500000000000001|
|1084.0|         2.8925129E7|0.6500000000000001|
|1084.0|         2.3847549E7|0.6500000000000001|
|1084.0|         1.1899362E7|0.6500000000000001|
|1084.0|1.291034620667072...|0.6500000000000001|
|1084.0|7.823882201894051...|0.6500000000000001|
|1084.0|8.135622507955568...|0.6500000000000001|
|1084.0|       2.796942515E9|0.6500000000000001|
|1084.0|1.303337342489550...|0.6500000000000001|
|1084.0|1.124354371255296E18|0.6500000000000001|
|1084.0|1.259375214112243...|0.6500000000000001|
|1084.0|         8.9421286E7|0.6500000000000001|
|1084.0|         1.3198502E7|0.6500000000000001|
|1084.0|8.076487532974121E17|0.6500000000000001|
|1084.0|         4.8473936E8|0.6500000000000001|
|1084.0|        7.28715229E8|0.6500000000000001|
|1084.0|         8.1

In [31]:
## Check the number of edges of each vertex
g.degrees.show(1000)

+--------------------+------+
|                  id|degree|
+--------------------+------+
|7.582774084116521E17|     1|
|9.386755863728128E17|     1|
|1.073288170433888...|     1|
|7.468350793018490...|     1|
|1.266874365137743...|     1|
|1.265386533252710...|     2|
|1.231268015146012...|     1|
|1.290069660914847...|     2|
|1.300875516992516...|     1|
|1.305239324833837...|     1|
|1.304852058458259...|     1|
|1.226185683016044...|     1|
|1.222722958977683...|     1|
|1.160979637981331...|     1|
|1.302335325465964...|     1|
|       1.647088994E9|     1|
|        1.69520068E8|     1|
|       2.858548343E9|     1|
|        3.08735874E8|     1|
|         2.4125959E7|     1|
|           5902792.0|     1|
|9.138175140559584E17|     1|
|       2.759425505E9|     1|
|           6.38158E7|     1|
|       2.786354775E9|     1|
|       1.853290633E9|     1|
|        2.37605275E8|     1|
|         1.8314252E7|     1|
|       3.334545087E9|     1|
|7.632606131579781...|     1|
|        2

In [32]:
inDegreeDF=g.inDegrees

In [33]:
inDegreeDF.sort(['inDegree'],ascending=[0]).show()

+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|         3.3923443E7|      21|
|       2.872661847E9|      16|
|         2.4733117E7|      11|
|         1.8082945E7|      10|
|         3.5767916E7|      10|
|        4.59269265E8|      10|
|         2.0414217E7|       9|
|         1.8539631E7|       9|
|7.789337531502510...|       9|
|         1.7681513E7|       9|
|              1688.0|       9|
|         1.4993272E7|       9|
|        2.42106743E9|       9|
|8.922758577791180...|       9|
|          1.680158E7|       9|
|         1.4855279E7|       9|
|         7.3986605E7|       9|
|8.860328770115584E17|       9|
|        1.79732982E8|       8|
|         4.8935222E7|       8|
+--------------------+--------+
only showing top 20 rows



In [34]:
PageRankResults = g.pageRank(resetProbability=0.15, tol=0.01)

In [35]:
PageRankResults.vertices.sort(['pagerank'],ascending=[0]).show(50)

+--------------------+-----+------------------+
|               Nodes|   id|          pagerank|
+--------------------+-----+------------------+
|         8.7035829E7| 1688|1.0041826374629281|
|       1.496850486E9| 3382|1.0028356359674973|
|        4.67768114E8|43313|1.0027919516046906|
|         1.8904259E7|12330| 1.002621982678155|
|1.265788105946251...|80523|  1.00174140488254|
|         1.6437255E7|79543| 1.001650669627907|
|       3.241154353E9| 2426|1.0015936954330427|
|       1.062214914E9| 3037|1.0014237265065076|
|1.303516156628799...|21493|1.0014237265065076|
|1.290120400974815...|13222|1.0014237265065076|
|       3.180963725E9|45643|1.0014237265065076|
|       1.418307036E9|45463|1.0014237265065076|
|       3.224901889E9| 2618|1.0014237265065076|
|1.304803011894665...|13258|1.0014237265065076|
|        5.78897107E8|34663|1.0014237265065076|
|1.004280647471509...|16623|1.0014237265065076|
|         8.5014955E7|33383|1.0014237265065076|
|8.739379789467197...|12294|1.0014237265

In [36]:
PageRankResults.edges.show(50)

+------+--------------------+------------------+------+
|   src|                 dst|      relationship|weight|
+------+--------------------+------------------+------+
|1084.0|           8227822.0|0.6500000000000001|2.0E-4|
|1084.0|         1.4630872E7|0.6500000000000001|2.0E-4|
|1084.0|         1.8359716E7|0.6500000000000001|2.0E-4|
|1084.0|          2.036159E7|0.6500000000000001|2.0E-4|
|1084.0|          4.576131E7|0.6500000000000001|2.0E-4|
|1084.0|         7.2192524E7|0.6500000000000001|2.0E-4|
|1084.0|        1.40647632E8|0.6500000000000001|2.0E-4|
|1084.0|        1.42982624E8|0.6500000000000001|2.0E-4|
|1084.0|        2.44160286E8|0.6500000000000001|2.0E-4|
|1084.0|        2.60399545E8|0.6500000000000001|2.0E-4|
|1084.0|        3.21788327E8|0.6500000000000001|2.0E-4|
|1084.0|        9.32428458E8|0.6500000000000001|2.0E-4|
|1084.0|        1.51125039E9|0.6500000000000001|2.0E-4|
|1084.0|       1.528252346E9|0.6500000000000001|2.0E-4|
|1084.0|        1.55664422E9|0.6500000000000001|

In [37]:
result = g.labelPropagation(maxIter=5)

result.sort(['label'],ascending=[0]).show()

+--------------------+-----+-------------------+
|               Nodes|   id|              label|
+--------------------+-----+-------------------+
|1.120168892498882...| 5242|1238979653492187136|
|         8.7035829E7| 1688| 823209839434338304|
|1.133589302070222...| 1084| 804226890999414784|
|        8.63008374E8|12514|         4853564932|
|         6.8687595E7| 2405|          933248257|
|        1.85706066E9|13476|          169983584|
|        1.06982465E8|11448|          161439462|
|        1.65164256E8| 5658|          115964963|
|       3.092806187E9|12924|           81277570|
|8.517348897413406...|23993|           20189103|
|        4.65046008E8| 1077|           19672966|
|1.085720909879619...|10326|           19413876|
|       4.892685173E9|12181|           19152095|
|       3.875569163E9|13351|           19148087|
|           2.20287E7| 3554|           18878927|
|         2.1686064E7|82363|           18878927|
|         1.0500412E7| 6136|           18365203|
|9.630898414906613..

In [38]:
result.sort(['label'],ascending=[0]).show(50)

+--------------------+------+-------------------+
|               Nodes|    id|              label|
+--------------------+------+-------------------+
|1.120168892498882...|  5242|1238979653492187136|
|         8.7035829E7|  1688| 823209839434338304|
|1.133589302070222...|  1084| 804226890999414784|
|        8.63008374E8| 12514|         4853564932|
|         6.8687595E7|  2405|          933248257|
|        1.85706066E9| 13476|          169983584|
|        1.06982465E8| 11448|          161439462|
|        1.65164256E8|  5658|          115964963|
|       3.092806187E9| 12924|           81277570|
|8.517348897413406...| 23993|           20189103|
|        4.65046008E8|  1077|           19672966|
|1.085720909879619...| 10326|           19413876|
|       4.892685173E9| 12181|           19152095|
|       3.875569163E9| 13351|           19148087|
|         2.1686064E7| 82363|           18878927|
|           2.20287E7|  3554|           18878927|
|         1.0500412E7|  6136|           18365203|


In [ ]:
result = g.connectedComponents()
result.show()

In [41]:
community_df = result.toPandas()

In [42]:
community_df

,Nodes,id,label
0,5.924463e+07,26,26
1,2.556428e+09,29,29
2,8.505984e+17,474,474
3,9.888948e+17,964,964
4,2.988842e+07,1677,1677
...,...,...,...
232557,1.274790e+18,230475,230475
232558,2.390727e+08,231317,231317
232559,1.196359e+18,231657,231657
232560,1.971308e+08,231840,231840


In [45]:
community_df.to_csv('/content/drive/My Drive/Twitter-Dataset-Analysis/295B/Nov09/Communities_Detected_For_FakeNews.csv',index=False)